# Hacker News Analysis: Does "Ask HN:" or "Show HN:" perform better?



This is an analysis performed on the site Hacker News (https://news.ycombinator.com/) to determine how certain posts perform. This dataset contains about 20,000 entries that came from a random sampling of 300,000. Link to the data set: https://www.kaggle.com/hacker-news/hacker-news-posts. 

In [49]:
import csv 

In [50]:
opened_file = open("hacker_news.csv")
reader = csv.reader(opened_file)
hn = list(reader)
headers = hn[:1]
hn = hn[1:]

print(hn[:5]) #first 5 rows

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


After taking a look at this data we see the columns are: **"id"**, **"title"**, **"url"**, **"num_points"**, **"num_comments"**, **"author"** and **"created_at"**. Note that created_at is in EST or Eastern Time.

In [51]:
print(headers)

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']]


The next steps are to seperate "Ask HN:" and "Show HN:" and other posts into seperate lists for analyzing.

In [52]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn: 
    title = row[1]
    title = title.lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))            
        

1744
1162
17194


We can see that ask posts outnumber show posts by a bit.

In [53]:
print(len(ask_posts) - len(show_posts))

582


Let's take a look at the first 5 posts of each list.

In [54]:
first_5_ask = ask_posts[:5]
first_5_show = show_posts[:5]

print("Ask Posts")
print("\n")
print(first_5_ask)
print("\n")
print("Show Posts")
print("\n")
print(first_5_show)

Ask Posts


[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14'], ['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20'], ['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']]


Show Posts


[['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46'], ['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https

One way to guage performance of a post is to look at the average number of comemnts. Let's take a look at the average amount of comments for the "Ask HN" posts.

In [55]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments = num_comments + total_ask_comments
    number_comments = len(ask_posts)

avg_ask_comments = total_ask_comments / number_comments

print(avg_ask_comments)
print(total_ask_comments)

14.038417431192661
24483


It appears that on average, the number of comments for "Ask HN:" posts is around **14**. The total number of comments in the "ask_posts" list is **24,483**.

Let's take a look on the average amount of comments for "Show HN:" posts.

In [56]:
total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments = num_comments + total_show_comments
    number_comments = len(show_posts)

avg_show_comments = total_show_comments / number_comments

print(avg_show_comments)
print(total_show_comments)

10.31669535283993
11988


It appears that on average, the number of comments for "Show HN:" posts is around **10**. The total number of comments in the "show_posts" list is **11,988**.

Now looking at the results we can see that the "Ask HN:" posts get more comments than "Show HN:" posts. This makes sense because "Ask HN:" posts are specifically **looking for answers for the questions they have**. "Show HN:" posts tend to be lower because the title isn't actively asking for engagement by the HN community.

## Looking at timing of posts

One factor to take in is the timing of the posts. Since "Ask HN:" gets more comments, let us focus on that moving forward. 

Could the timing of the post attract more comments? Let's take a look and see how many posts are created in each hour of the day, along with the number of comments.

After that, we will calculate average number of comments ask posts receive by the hour they're created.

In [57]:
import datetime as dt

result_list = []

for post in ask_posts:
    result_list.append([post[6], int(post[4])])


counts_by_hour = {} #the number of posts by the hour
comments_by_hour = {} #the number of comments by the hour
date_format = "%m/%d/%Y %H:%M"

for each_row in result_list:
    date = each_row[0]
    comment = each_row[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    if time not in counts_by_hour:
        counts_by_hour[time] = 1
        comments_by_hour[time] = comment
    else:
        counts_by_hour[time] += 1
        comments_by_hour[time] += comment

print(counts_by_hour)
print("\n")
print(comments_by_hour)

{'00': 55, '10': 59, '22': 71, '01': 60, '13': 85, '05': 46, '21': 109, '03': 54, '12': 73, '04': 47, '14': 107, '08': 48, '09': 45, '18': 109, '16': 108, '07': 34, '11': 58, '17': 100, '02': 58, '23': 68, '15': 116, '20': 80, '06': 44, '19': 110}


{'00': 447, '10': 793, '22': 479, '01': 683, '13': 1253, '05': 464, '21': 1745, '03': 421, '12': 687, '04': 337, '14': 1416, '08': 492, '09': 251, '18': 1439, '16': 1814, '07': 267, '11': 641, '17': 1146, '02': 1381, '23': 543, '15': 4477, '20': 1722, '06': 397, '19': 1188}


You can see we have a breakdown of each hour (in 24-hour format) and the number of counts (in the first dictionary) or number of comments (second dictionary). We would then divide the number of comments by posts to find the average amount of comements "Ask HN:" posts would receive at each hour of the day.

In [58]:
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr] / counts_by_hour[hr]])
    
avg_by_hour

[['00', 8.127272727272727],
 ['10', 13.440677966101696],
 ['22', 6.746478873239437],
 ['01', 11.383333333333333],
 ['13', 14.741176470588234],
 ['05', 10.08695652173913],
 ['21', 16.009174311926607],
 ['03', 7.796296296296297],
 ['12', 9.41095890410959],
 ['04', 7.170212765957447],
 ['14', 13.233644859813085],
 ['08', 10.25],
 ['09', 5.5777777777777775],
 ['18', 13.20183486238532],
 ['16', 16.796296296296298],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034],
 ['17', 11.46],
 ['02', 23.810344827586206],
 ['23', 7.985294117647059],
 ['15', 38.5948275862069],
 ['20', 21.525],
 ['06', 9.022727272727273],
 ['19', 10.8]]

We have the results, but let's clean it up so we can see the top 5 hours to post based on comments.

In [59]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

print(swap_avg_by_hour)
print("\n")

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print("Top 5 Hours for Ask Posts Comments")
print("\n")
for avg, hr in sorted_swap[:5]:
    print(
        "{}: {:.2f} average comments per post".format(
            dt.datetime.strptime(hr, "%H").strftime("%H:%M"), avg
        )
    )
    

[[8.127272727272727, '00'], [13.440677966101696, '10'], [6.746478873239437, '22'], [11.383333333333333, '01'], [14.741176470588234, '13'], [10.08695652173913, '05'], [16.009174311926607, '21'], [7.796296296296297, '03'], [9.41095890410959, '12'], [7.170212765957447, '04'], [13.233644859813085, '14'], [10.25, '08'], [5.5777777777777775, '09'], [13.20183486238532, '18'], [16.796296296296298, '16'], [7.852941176470588, '07'], [11.051724137931034, '11'], [11.46, '17'], [23.810344827586206, '02'], [7.985294117647059, '23'], [38.5948275862069, '15'], [21.525, '20'], [9.022727272727273, '06'], [10.8, '19']]


Top 5 Hours for Ask Posts Comments


15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


As you can see the hour of the day that on average gets the **most points is 1500 or 3pm**. The **second highest is 2am** and **third highest would be 8pm**. 

In summary, it appears that the most active time for "Ask HN:" posts based on the data set is 3pm EST. If I were to ensure my "Ask HN:" post got the most amount of comments then I would make sure to post around 3pm-4pm and if I had to choose a second time it would be around 8pm EST. 

Although 2am CST has the second highest comments per post, I would not recommend staying up that late! :wink: